# D-LinOSS Realtime Demo (egui)

This notebook launches a small egui window that animates a D-LinOSS layer running on CPU in realtime.

Use the single :dep pattern and enable gui, fft, and audio features to hear a gentle Moog-ish sweep for the green input.

In [5]:
:dep dlinoss-notebooks = { path = ".", features = ["gui", "audio"] }

// Initialize portable working directory for assets
use dlinoss_notebooks::{set_notebook_cwd, set_image_store_rel_dir};
set_notebook_cwd().expect("failed to set notebook cwd");
set_image_store_rel_dir("images_store").ok();

println!("✅ D-LinOSS Notebooks Environment Ready (No Zip Issues!)");
println!("CWD: {}", std::env::current_dir().unwrap().display());

✅ D-LinOSS Notebooks Environment Ready (No Zip Issues!)
CWD: /home/rustuser/projects/rust/active/dlinossrustcandle/notebooks
CWD: /home/rustuser/projects/rust/active/dlinossrustcandle/notebooks


## Launch the realtime window + audio

The window runs independently; close it to return to the notebook. Enable your speakers/headphones to hear the green input as a gentle Moog-ish sweep.

In [6]:
use dlinoss_notebooks::anyhow; // re-exported anyhow
use dlinoss_notebooks::{DLinOssLayer, DLinOssLayerConfig, Device, DType, Tensor};
use dlinoss_notebooks::realtime_gui::{run_realtime_demo_with_audio, TensorProcessor};

// Minimal processor that runs the DLinOSS layer on incoming tensors
struct DlinossProcessor {
    layer: DLinOssLayer,
}

impl TensorProcessor for DlinossProcessor {
    fn process(&mut self, input: &Tensor) -> anyhow::Result<Tensor> {
        // Forward expects shape [B, T, In]; TinyApp provides that
        Ok(self.layer.forward(input, None)?)
    }
}

(|| -> anyhow::Result<()> {
    // Set up a small DLinOSS layer on CPU
    let dev = Device::Cpu;
    let cfg = DLinOssLayerConfig {
        state_dim: 8,
        input_dim: 1,
        output_dim: 1,
        delta_t: 1e-2,
        dtype: DType::F32,
    };
    let layer = DLinOssLayer::new(cfg, &dev)?;
    let proc = DlinossProcessor { layer };

    // Launch the GUI + audio demo with our processor
    run_realtime_demo_with_audio(proc)?;
    println!("done (audio)");
    Ok(())
})().unwrap_or_else(|e| eprintln!("error: {e}"));


done (audio)


In [ ]:
/// Notebook helper: CPU-only forward pass check.
/// Runs a small sine input through DLinOssLayer and prints y dims.
use dlinoss_notebooks::*;            // Tensor, Device, DType, SignalGen, DLinOssLayer, DLinOssLayerConfig
use dlinoss_notebooks::anyhow;       // anyhow::Result for closures

match (|| -> anyhow::Result<()> {
    let dev = Device::Cpu;
    let cfg = DLinOssLayerConfig {
        state_dim: 8,
        input_dim: 1,
        output_dim: 1,
        delta_t: 1e-2,          // f64
        dtype: DType::F32,
    };
    let layer = DLinOssLayer::new(cfg, &dev)?;

    // A short sine burst: shape [1, 128, 1]
    let x = SignalGen::sine(128, 2.0, 1e-2)?;
    let y = layer.forward(&x, None)?;
    println!("y dims: {:?}", y.dims());
    Ok(())
})() {
    Ok(()) => println!("CPU-only forward pass: ok"),
    Err(e) => eprintln!("CPU-only forward pass error: {e}"),
}


y dims: [1, 128, 1]
CPU-only forward pass: ok
CPU-only forward pass: ok


()

: 